# Purpose
The purpose of this notebook is to retreive and pre-process the GIPL 2.0 model outputs described by the following publication:

<i>
Climate damages to Alaska public infrastructure
April M. Melvin, Peter Larsen, Brent Boehlert, James E. Neumann, Paul Chinowsky, Xavier Espinet, Jeremy Martinich, Matthew S. Baumann, Lisa Rennels, Alexandra Bothner, Dmitry J. Nicolsky, Sergey S. Marchenko
Proceedings of the National Academy of Sciences Jan 2017, 114 (2) E122-E131; DOI: 10.1073/pnas.1611056113
</i>

The goal is ingest the data as a netCDF to Rasdaman and then create an endpoint around the data in the SNAP Data API for use in the [Northern Climate Reports](https://northerclimatereports.org) app.


## Background and Motivation
Data represent RCP 4.5 and 8.5 scenarios across 5 climate models. Outputs are binned into "eras" and there are two GIPL permafrost variables Active Layer Thickness (ALT) and Mean Annual Ground Temperature (MAGT).
The goal is to create a completely inclusive datacube of both historical and projected data. 
It will have the following dimensions for both ALT and MAGT variables:
* era
* model
* scenario
* Y
* X

# Steps
 - Download data from the [Google Drive folder shared by Jeremy Littell](https://drive.google.com/file/d/1NaZInm20tpnv63NcC5_p3xTWp5gC3Ty6/view?usp=sharing]) to the `input_data` directory.
 - Convert the ASCII raster data to GeoTIFF using the accompanying projection information contained in the conjugate .prj file. There is one .prj file for each ASCII file, but they are all the same.
 - Encode era, climate model, and emission scenario information in the GeoTIFF file name.
 - Crop the GeoTiffs to a [known Alaskan spatial boundary](https://github.com/ua-snap/geospatial-vector-veracity/tree/main/vector_data/polygon/boundaries/alaska_coast_simplified)
 - Convert the GeoTIFF stack to a multidimensional netCDF for ingest to Rasdaman.
 

## References
 - [Melvin et al., 2017Z](https://doi.org/10.1073/pnas.1611056113)
   - [Supporting Information](https://www.pnas.org/content/pnas/suppl/2016/12/21/1611056113.DCSupplemental/pnas.201611056SI.pdf?targetid=nameddest%3DSTXT)
 - [README: ASCII file naming convention key (J. Littel)](https://docs.google.com/document/d/1OdoOqu8pFLnjbyWC_0f5ev8fS-ilZ3pD/edit#heading=h.gjdgxs) 



In [1]:
ls input_data/Permafrost/Projections

ALT_10_1.asc  ALT_16_3.prj  ALT_6_2.asc    MAGT_13_2.prj  MAGT_2_3.asc
ALT_10_1.prj  ALT_16_4.asc  ALT_6_2.prj    MAGT_13_3.asc  MAGT_2_3.prj
ALT_10_2.asc  ALT_16_4.prj  ALT_6_3.asc    MAGT_13_3.prj  MAGT_2_4.asc
ALT_10_2.prj  ALT_17_1.asc  ALT_6_3.prj    MAGT_13_4.asc  MAGT_2_4.prj
ALT_10_3.asc  ALT_17_1.prj  ALT_6_4.asc    MAGT_13_4.prj  MAGT_3_1.asc
ALT_10_3.prj  ALT_17_2.asc  ALT_6_4.prj    MAGT_1_3.asc   MAGT_3_1.prj
ALT_10_4.asc  ALT_17_2.prj  ALT_7_1.asc    MAGT_1_3.prj   MAGT_3_2.asc
ALT_10_4.prj  ALT_17_3.asc  ALT_7_1.prj    MAGT_14_1.asc  MAGT_3_2.prj
ALT_11_1.asc  ALT_17_3.prj  ALT_7_2.asc    MAGT_14_1.prj  MAGT_3_3.asc
ALT_11_1.prj  ALT_17_4.asc  ALT_7_2.prj    MAGT_14_2.asc  MAGT_3_3.prj
ALT_11_2.asc  ALT_17_4.prj  ALT_7_3.asc    MAGT_14_2.prj  MAGT_3_4.asc
ALT_11_2.prj  ALT_18_1.asc  ALT_7_3.prj    MAGT_14_3.asc  MAGT_3_4.prj
ALT_11_3.asc  ALT_18_1.prj  ALT_7_4.asc    MAGT_14_3.prj  MAGT_4_1.asc
ALT_11_3.prj  ALT_18_2.asc  ALT_7_4.prj    MAGT_14_4.asc  MAGT_4_1.prj
ALT_11

In [1]:
from pathlib import Path
import numpy as np
import xarray as xr
import pandas as pd
import rasterio as rio
import os

In [7]:
# Verify the CRS of the ASCII data and the general shape of each file
ascii_proj = %cat input_data/Permafrost/Projections/MAGT_10_1.prj
header = !head -n 6 input_data/Permafrost/Projections/MAGT_10_1.asc
src = rio.open('input_data/Permafrost/Projections/MAGT_10_1.asc')
meta = src.meta
profile = src.profile
shape = src.read(1).shape
ascii_proj

PROJCS["Alaska_Albers_Equal_Area_Conic",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Albers"],PARAMETER["False_Easting",0],PARAMETER["False_Northing",0],PARAMETER["central_meridian",-154],PARAMETER["Standard_Parallel_1",55],PARAMETER["Standard_Parallel_2",65],PARAMETER["latitude_of_origin",50],UNIT["Meter",1]]

In [8]:
header

['ncols     1609',
 'nrows     593',
 'xllcenter -2172223.20581',
 'yllcenter 177412.93264',
 'cellsize  4000',
 'NODATA_value -9999']

In [10]:
print(shape)
print(profile)

(593, 1609)
{'driver': 'AAIGrid', 'dtype': 'float32', 'nodata': -9999.0, 'width': 1609, 'height': 593, 'count': 1, 'crs': CRS.from_wkt('PROJCS["Alaska_Albers_Equal_Area_Conic",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",50],PARAMETER["longitude_of_center",-154],PARAMETER["standard_parallel_1",55],PARAMETER["standard_parallel_2",65],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(4000.0, 0.0, -2174223.20581,
       0.0, -4000.0, 2547412.9326400002), 'tiled': False}


Wow! Rasterio knows where to look when the file names of the .prj and .asc files have indentical prefixes. These data appear to be in uniform shape and rasterio "knows" how to handle them. No red flags here. Per the README doc referenced earlier, we'll need a look-up table to retain era / model / scenario info and to generate output filenames.

```
These asci files are labeled by Variable_<Model/Policy Combination>_<Era>

Model-Scenario Combinations (10-19)
10: MRI-CGCM3, rcp4.5
11: MRI-CGCM3, rcp8.5
12: IPSL-CM5A-LR, rcp4.5
13: IPSL-CM5A-LR, rcp8.5
14: GISS-E2-R, rcp 4.5
15: GISS-E2-R, rcp 8.5
16: GFDL-CM3, rcp4.5
17: GFDL-CM3, rcp8.5
18: CCSM4, rcp4.5
19: CCSM4, rcp8.5
Eras (1-4): 30 year eras surrounding the following center dates, except last era is truncated at 2100.
1: 2025 (2011 - 2040)
2: 2050 (2036 - 2065)
3: 2075 (2061 – 2090)
4: 2095 (2086 – 2100)

The historical baseline is 1986-2005.
```

In [12]:
model_scenario_di = {10: "MRICGCM3_rcp45",
                     11: "MRICGCM3_rcp85",
                     12: "IPSLCM5ALR_rcp45",
                     13: "IPSLCM5ALR_rcp85",
                     14: "GISSE2R_rcp45",
                     15: "GISSE2R_rcp85",
                     16: "GFDLCM3_rcp45",
                     17: "GFDLCM3_rcp85",
                     18: "NCARCCSM4_rcp45",
                     19: "NCARCCSM4_rcp85",
                    }
era_di = {1: "era2025_2011to2040",
          2: "era2050_2036to2065",
          3: "era2075_2061to2090",
          4: "era2095_2086to2100",
         }

In [15]:
# Grab all the file paths for PROJECTED data (historical data (n=2) are in a separate directory)
target_dir = Path("input_data/Permafrost/Projections/")
out_dir = Path("input_data/Permafrost/geotiff/")
asc_fps = [fp.name for fp in target_dir.glob("*_*.asc")]
cmip5_asc_fps = [target_dir.joinpath(x) for x in asc_fps if int(x.split("_")[1]) >= 10]
n_cmip5_ascs = len(cmip5_asc_fps)
print(n_cmip5_ascs)

80


In [16]:
def make_new_filenames(output_dst, fp_list):
    """
    Blapping together new filenames of the format variable_model_scenario_era.tif
    """
    new_fps = []
    for fp in fp_list:
        pf_var, model_scenario_key, era_key = fp.name[:-4].split("_")
        new_fp = pf_var.lower() + "_" + model_scenario_di[int(model_scenario_key)].lower() + "_" + era_di[int(era_key)] + ".tif"
        new_fp = output_dst.joinpath(new_fp)
        new_fps.append(new_fp)
    return new_fps


def read_raster(raster_fp):
    """
    Read raster to numpy array.
    Read a single raster with rasterio and store raster in memory as a numpy
    array. Also reads and returns the profile which includes metadata and the neccessary information
    to write congruent datasets.
    Args:
        raster_fp (str): filepath to raster
    Returns:
        arr (ndarray): array of raster values
        profile (dict): metadata profile
    """

    src = rio.open(raster_fp)
    arr = src.read(1)
    profile = src.profile
    return (arr, profile)


def write_raster(arr, outpath, profile):
    """
    Write numpy array to disk as a raster with correct metadata.
    Args:
        arr (ndarray): array of raster values
        outpath (str): output filename and path for raster
        profile (dict): metadata for output
    Returns: None
    """

    with rio.open(outpath, 'w', **profile) as dst:
        dst.write(arr, 1)

In [21]:
geotiff_fps = make_new_filenames(out_dir, cmip5_asc_fps)
print(geotiff_fps[0])
print(geotiff_fps[44])
print(geotiff_fps[-1])

input_data/Permafrost/geotiff/alt_mricgcm3_rcp45_era2025_2011to2040.tif
input_data/Permafrost/geotiff/magt_mricgcm3_rcp85_era2025_2011to2040.tif
input_data/Permafrost/geotiff/magt_ncarccsm4_rcp85_era2095_2086to2100.tif


In [24]:
# convert the ASCI data to geotiff
#!mkdir input_data/Permafrost/geotiff
for asc, gtiff in zip(cmip5_asc_fps, geotiff_fps):
    arr, profile = read_raster(asc)
    profile.update(driver="GTiff", compress="lzw")
    write_raster(arr, gtiff, profile)

In [26]:
ls input_data/Permafrost/geotiff

alt_gfdlcm3_rcp45_era2025_2011to2040.tif
alt_gfdlcm3_rcp45_era2050_2036to2065.tif
alt_gfdlcm3_rcp45_era2075_2061to2090.tif
alt_gfdlcm3_rcp45_era2095_2086to2100.tif
alt_gfdlcm3_rcp85_era2025_2011to2040.tif
alt_gfdlcm3_rcp85_era2050_2036to2065.tif
alt_gfdlcm3_rcp85_era2075_2061to2090.tif
alt_gfdlcm3_rcp85_era2095_2086to2100.tif
alt_gisse2r_rcp45_era2025_2011to2040.tif
alt_gisse2r_rcp45_era2050_2036to2065.tif
alt_gisse2r_rcp45_era2075_2061to2090.tif
alt_gisse2r_rcp45_era2095_2086to2100.tif
alt_gisse2r_rcp85_era2025_2011to2040.tif
alt_gisse2r_rcp85_era2050_2036to2065.tif
alt_gisse2r_rcp85_era2075_2061to2090.tif
alt_gisse2r_rcp85_era2095_2086to2100.tif
alt_ipslcm5alr_rcp45_era2025_2011to2040.tif
alt_ipslcm5alr_rcp45_era2050_2036to2065.tif
alt_ipslcm5alr_rcp45_era2075_2061to2090.tif
alt_ipslcm5alr_rcp45_era2095_2086to2100.tif
alt_ipslcm5alr_rcp85_era2025_2011to2040.tif
alt_ipslcm5alr_rcp85_era2050_2036to2065.tif
alt_ipslcm5alr_rcp85_era2075_2061to2090.tif
alt_ipslcm5alr_rcp85_era2095_2086to2

These outputs look good - but we still have to process the historical data.

In [28]:
historical_suffix = "cruts31_historical_era1995_1986to2005.tif"
target_dir = Path("input_data/Permafrost/Base/")
out_dir = Path("input_data/Permafrost/geotiff/")
hist_asc_fps = [target_dir.joinpath(fp.name) for fp in target_dir.glob("*.asc")]
out_names = [out_dir.joinpath("magt_" + historical_suffix),
             out_dir.joinpath("alt_" + historical_suffix)]
print(out_names)

[PosixPath('input_data/Permafrost/geotiff/magt_cruts31_historical_era1995_1986to2005.tif'), PosixPath('input_data/Permafrost/geotiff/alt_cruts31_historical_era1995_1986to2005.tif')]


The convention is `variable_model_scenario_era_yearRange` although it is a little weird here because for the two historical files we are calling CRU TS 3.1 a "model" and "historical" a scenario. That is just the way it is when jamming together historical baselines and projected futures.

In [30]:
for asc, gtiff in zip(hist_asc_fps, out_names):
    arr, profile = read_raster(asc)
    profile.update(driver="GTiff", compress="lzw")
    write_raster(arr, gtiff, profile)

In [31]:
ls input_data/Permafrost/geotiff | grep historical

alt_cruts31_historical_era1995_1986to2005.tif
magt_cruts31_historical_era1995_1986to2005.tif


OK we have succesfully assembled 82 GeoTIFFs from the ASCII raster data. The last move here is to force these to a known Alaskan spatial extent. The data do not cover Canada anyway, so this doesn't reduce the value of the dataset, but will just help it mesh with our other data holdings. We could this on the first creation of the GeoTIFFs but I'll just the use the GDAL one liner.

In [33]:
ls input_data/Permafrost/clipper_shp/

Alaska_Coast_Simplified_Polygon.dbf  Alaska_Coast_Simplified_Polygon.shp.xml
Alaska_Coast_Simplified_Polygon.prj  Alaska_Coast_Simplified_Polygon.shx
Alaska_Coast_Simplified_Polygon.shp


Execute the crop script to use the above shapefile to bound the raster extent.

In [2]:
# Now that the GeoTIFFs are prepared, we can move on to creating a single netCDF
data_dir = Path("input_data/Permafrost/geotiff/cropped/")
data_fps = sorted(data_dir.glob("*"))
data_fps[0]

PosixPath('input_data/Permafrost/geotiff/cropped/alt_cruts31_historical_era1995_1986to2005.tif')

In [12]:
# variables needed to describe the data
varnames = ["magt", "alt"]
scenarios = ["historical", "rcp45", "rcp85"]
models = ["cruts31", "gfdlcm3", "gisse2r", "ipslcm5alr", "mricgcm3", "ncarccsm4"]
eras = ["1995", "2025", "2050", "2075", "2095"]
era_starts = ["1986", "2011", "2036", "2061", "2086"]
era_ends = ["2005", "2040", "2065", "2090", "2100"]
units_lu = {"magt": "°C", "alt": "m"}

# integer encoding for strings for the netcdf coords (Rasdaman wants this)
era_encoding = {"1995": 0, "2025": 1, "2050": 2, "2075": 3, "2095": 4}
model_encoding = {"cruts31": 0, "gfdlcm3": 1, "gisse2r": 2, "ipslcm5alr": 3, "mricgcm3": 4, "ncarccsm4": 5}
scenario_encoding = {"historical": 0, "rcp45": 1, "rcp85": 2}
all_encoding = {**units_lu, **era_encoding, **scenario_encoding, **model_encoding}

# get x and y dimensions from a single file
with rio.open(data_fps[0]) as src:
    src_meta = src.meta.copy()
    # get x and y coordinates for axes
    y = np.array([src.xy(i, 0)[1] for i in np.arange(src.height)])
    x = np.array([src.xy(0, j)[0] for j in np.arange(src.width)])
    # get the number of pixels
    ny, nx = src.height, src.width 

In [13]:
# creating a dictionary from all the raster files
# the directory of rasters to dict is kind of boilerplate at this point
# key is the filename, value is a subdictionary with keys for each characteristic
# we'll force -9999.0 as the no data value for good measure.
data_di = {}

for fp in data_fps:
    fn = fp.name.split(".tif")[0]
    data_di[fn] = {}
    fn_components = fn.split("_")
    data_di[fn]["varname"] = fn_components[0]
    data_di[fn]["model"] = fn_components[1]
    data_di[fn]["scenario"] = fn_components[2]
    data_di[fn]["era"] = fn_components[3][-4:]
    data_di[fn]["era start"] = fn_components[4][0:4]
    data_di[fn]["era end"] = fn_components[4][-4:]
    
    with rio.open(fp) as src:
    
        arr = src.read(1)
        arr[np.isnan(arr)] = -9999.0
        data_di[fn]["arr"] = arr

data_di['alt_gfdlcm3_rcp45_era2025_2011to2040']

{'varname': 'alt',
 'model': 'gfdlcm3',
 'scenario': 'rcp45',
 'era': '2025',
 'era start': '2011',
 'era end': '2040',
 'arr': array([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        ...,
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
        [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],
       dtype=float32)}

### Higher Dimensions
This is where it gets interesting. We need to define the shape of our data cube on a per variable basis. In this instance that'll be era X model X scenario X x-coordinate X y-coordinate. That's a 5 dimensional *hypercube* for those scoring at home (space (x and y) plus time (era) plus model and scenario). There may be a simpler way to do this, but setting up arrays full of no data (e.g. -9999) is a good start and will act as governor when it comes to pushing data because if we exceed the indicies of the array, numpy will yell at us. It is also a memory check - but that shouldn't be an issue on Apollo / Zeus.

In [14]:
# set up a multidimensional array
arr_shape = (len(eras),
             len(models),
             len(scenarios),
             ny,
             nx)

out_arr = np.full(arr_shape, -9999.0, dtype=np.float32)
print(out_arr.shape)

(5, 6, 3, 489, 914)


This place-holder array checks out. 5 possible era, 6 possible models, 3 possible scenarios. Specifying `dtype` here is important. This should match the `dtype` of the input GeoTIFFs. We are not done initializing arrays though. The hypercube needs to get filled, even when data does not exist because of invalid dimensional combinations. For example, we have no "historical-ncarccsm4" scenario-model combinatiion GeoTIFF (because it is nonsense). But should create an array we can push to the hypercube for those indicies.

In [15]:
# set up a "null" array for invalid dimensional combos by grabbing a slice of the place-holder array
null_arr = out_arr[0, 0, 0,].copy()
print(null_arr.shape) 

(489, 914)


Now we convert the dictionary full of raster data to a DataFrame where each row is a file and columns reflect the data and the describing characteristics. I'm not convinced this step is totally necessary, but querying a dictionary, especially a nested dictionary, is sort of fraught. The DataFrame is a bit more friendly. 

In [16]:
df = pd.DataFrame.from_dict(data_di).sort_index().T
df

,arr,era,era end,era start,model,scenario,varname
alt_cruts31_historical_era1995_1986to2005,"[[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0,...",1995,2005,1986,cruts31,historical,alt
alt_gfdlcm3_rcp45_era2025_2011to2040,"[[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0,...",2025,2040,2011,gfdlcm3,rcp45,alt
alt_gfdlcm3_rcp45_era2050_2036to2065,"[[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0,...",2050,2065,2036,gfdlcm3,rcp45,alt
alt_gfdlcm3_rcp45_era2075_2061to2090,"[[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0,...",2075,2090,2061,gfdlcm3,rcp45,alt
alt_gfdlcm3_rcp45_era2095_2086to2100,"[[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0,...",2095,2100,2086,gfdlcm3,rcp45,alt
...,...,...,...,...,...,...,...
magt_ncarccsm4_rcp45_era2095_2086to2100,"[[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0,...",2095,2100,2086,ncarccsm4,rcp45,magt
magt_ncarccsm4_rcp85_era2025_2011to2040,"[[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0,...",2025,2040,2011,ncarccsm4,rcp85,magt
magt_ncarccsm4_rcp85_era2050_2036to2065,"[[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0,...",2050,2065,2036,ncarccsm4,rcp85,magt
magt_ncarccsm4_rcp85_era2075_2061to2090,"[[-9999.0, -9999.0, -9999.0, -9999.0, -9999.0,...",2075,2090,2061,ncarccsm4,rcp85,magt


This DataFrame checks out. Next a nested loop will populate a copy of the place-holder `out_arr` for each data variable (MAGT and ALT in this case). The key thing here is the ORDER. We have to be certain that we are iterating in sync with the shape of the place-holder array. We defined our output data structure so we have to stick to it. Era is the first (technically 0th) dimension, model is the second, and scenario the third.

In [17]:
out_arrs_by_var = []

for var in varnames:
    arr_to_fill = out_arr.copy()
    for era, er in zip(eras, range(out_arr.shape[0])):
        for model, mn in zip(models, range(out_arr.shape[1])):
            for scenario, sc in zip(scenarios, range(out_arr.shape[2])):
                query = "era == @era & scenario == @scenario & model == @model"
                try:
                    sub_arr = df[df.varname == var].query(query)["arr"].values[0]
                except IndexError:
                    sub_arr = null_arr.copy()
                arr_to_fill[er, mn, sc] = sub_arr
                
    out_arrs_by_var.append(arr_to_fill)

In [18]:
varnames

['magt', 'alt']

In [19]:
magt_arr = np.array(out_arrs_by_var[0])
alt_arr = np.array(out_arrs_by_var[1])
print(magt_arr.shape, alt_arr.shape)

(5, 6, 3, 489, 914) (5, 6, 3, 489, 914)


Looks good! A 5 dimensional array for each variable: 5 possible era, 6 possible models, 3 possible scenarios, Y, X. Now we'll create an xarray Dataset object and prescribe the dimensions. We'll use the integer encoding for the coordinate values to play nice with Rasdaman.

In [20]:
dim_names = ["era", "model", "scenario", "y", "x"]

ds = xr.Dataset(data_vars={"magt": (dim_names, magt_arr),
                           "alt": (dim_names, alt_arr)},
                coords={"era": [era_encoding[era] for era in eras],
                        "model": [model_encoding[model] for model in models],
                        "scenario": [scenario_encoding[scenario] for scenario in scenarios],
                        "y": y,
                        "x": x},
               attrs=all_encoding)

ds

<xarray.Dataset>
Dimensions:   (era: 5, model: 6, scenario: 3, y: 489, x: 914)
Coordinates:
  * era       (era) int64 0 1 2 3 4
  * model     (model) int64 0 1 2 3 4 5
  * scenario  (scenario) int64 0 1 2
  * y         (y) float64 2.373e+06 2.369e+06 2.365e+06 ... 4.254e+05 4.214e+05
  * x         (x) float64 -2.164e+06 -2.16e+06 ... 1.484e+06 1.488e+06
Data variables:
    magt      (era, model, scenario, y, x) float32 -9.999e+03 ... -9.999e+03
    alt       (era, model, scenario, y, x) float32 -9.999e+03 ... -9.999e+03
Attributes: (12/16)
    magt:        °C
    alt:         m
    1995:        0
    2025:        1
    2050:        2
    2075:        3
    ...          ...
    cruts31:     0
    gfdlcm3:     1
    gisse2r:     2
    ipslcm5alr:  3
    mricgcm3:    4
    ncarccsm4:   5

This is a quick test of the historical ALT data as read straight from the original GeoTIFF and as sliced from the cube to be sure they are identical.

In [26]:
test_slice = ds.sel(era=1, model=2, scenario=1).alt

In [27]:
test_slice

<xarray.DataArray 'alt' (y: 489, x: 914)>
array([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       ...,
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
       [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],
      dtype=float32)
Coordinates:
    era       int64 1
    model     int64 2
    scenario  int64 1
  * y         (y) float64 2.373e+06 2.369e+06 2.365e+06 ... 4.254e+05 4.214e+05
  * x         (x) float64 -2.164e+06 -2.16e+06 ... 1.484e+06 1.488e+06

In [28]:
print(type(test_slice.data))
print(test_slice.dtype)
print(test_slice.data.shape)

<class 'numpy.ndarray'>
float32
(489, 914)


In [29]:
src = rio.open("input_data/Permafrost/geotiff/cropped/alt_gisse2r_rcp45_era2025_2011to2040.tif")
test_arr = src.read(1)
print(src.meta)
print(type(test_arr))
print(test_arr.dtype)
print(test_arr.shape)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -9999.0, 'width': 914, 'height': 489, 'count': 1, 'crs': CRS.from_epsg(3338), 'transform': Affine(4000.0, 0.0, -2166223.20581,
       0.0, -4000.0, 2375412.9326400002)}
<class 'numpy.ndarray'>
float32
(489, 914)


In [30]:
(test_slice.data == test_arr).all()

True

In [31]:
# specify encoding to compress
encoding = {"magt": {"zlib": True, "complevel": 9, "_FillValue": -9999.0},
            "alt": {"zlib": True, "complevel": 9, "_FillValue": -9999.0},
           }

In [32]:
ds.to_netcdf("gipl_alt_magt_4km.nc", encoding=encoding)

In [33]:
ls -lhrt

total 33M
-rwxr-xr-x. 1 cparr4 snap_users 2.6K Nov 23 17:37 ingest.json*
-rw-r--r--. 1 cparr4 snap_users  323 Dec  7 13:24 download_melvin.sh
drwxr-xr-x. 3 cparr4 snap_users   46 Dec  7 14:13 input_data/
-rw-r--r--. 1 cparr4 snap_users 1006 Dec  7 14:42 download_ak_polygon.sh
-rw-r--r--. 1 cparr4 snap_users  252 Dec  9 07:41 crop_geotiff_dir_to_akpoly.sh
-rw-r--r--. 1 cparr4 snap_users  38K Dec  9 11:56 ingest.json.log
-rw-r--r--. 1 cparr4 snap_users  97K Dec 10 11:10 preprocess_GIPL_4km_alt_and_magt.ipynb
-rw-r--r--. 1 cparr4 snap_users  18M Dec 10 11:11 gipl_alt_magt_4km.nc
